<a href="https://colab.research.google.com/github/emmafengxi/Detect-Medicare-Fraud/blob/master/Test_with_TN_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# get access to google drive, where data are stored
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import  datetime
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#!pip install recordlinkage
#import recordlinkage as rl

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# 1. Clean Lable Data


---


> Latest updated in Aug 2019










In [3]:
# load LEIE data for TN
df_LEIE = pd.read_csv('/content/drive/My Drive/Data Science/Springboard/Data/LEIE_TN.csv', index_col=0)
df_LEIE.head(3)

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
227,NaN,NaN,,ARGUS MEDICAL SUPPLIES,HOME HEALTH AGENCY,NaN,NaN,0,NaN,434 PARK STREET,LEBANON,TN,37087,1128b8,19930704,0,0,NaN
378,NaN,NaN,,BLANKENSHIP'S PHARMACY,PHARMACY,NaN,NaN,0,NaN,106 HAYES STREET,CROSSVILLE,TN,33873,1128a1,19881011,0,0,NaN
482,NaN,NaN,,"CARESOUTH CLINIC, PC",MEDICAL GROUP,NaN,NaN,0,NaN,1203 VANN DRIVE,JACKSON,TN,38305,1128b7,20050722,0,0,NaN


In [0]:
df_LEIE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 227 to 72455
Data columns (total 18 columns):
LASTNAME      1655 non-null object
FIRSTNAME     1655 non-null object
MIDNAME       1691 non-null object
BUSNAME       36 non-null object
GENERAL       1691 non-null object
SPECIALTY     1631 non-null object
UPIN          125 non-null object
NPI           1691 non-null int64
DOB           1646 non-null float64
ADDRESS       1690 non-null object
CITY          1691 non-null object
STATE         1691 non-null object
ZIP           1691 non-null int64
EXCLTYPE      1691 non-null object
EXCLDATE      1691 non-null int64
REINDATE      1691 non-null int64
WAIVERDATE    1691 non-null int64
WVRSTATE      0 non-null float64
dtypes: float64(2), int64(5), object(11)
memory usage: 251.0+ KB


In [4]:
# extract the following exclusion types which are considered Medicare fradulent types: 
# '1128a1', '1128a2', '1128a3', '1128b4', '1128c3gi', '1128c3gii'
df_LEIE = df_LEIE[df_LEIE.EXCLTYPE.isin(['1128a1', '1128a2', '1128a3', '1128b4', '1128c3gi', '1128c3gii'])]

# create a provider type
df_LEIE['p_type'] = np.where((df_LEIE.LASTNAME.isna() & df_LEIE.BUSNAME.notna()), 'O', 'I')

# an organizational provider does not have first/last name, so move business name to last name
df_LEIE.LASTNAME = np.where((df_LEIE.LASTNAME.isna() & df_LEIE.BUSNAME.notna()), df_LEIE.BUSNAME, df_LEIE.LASTNAME)

# drop BUSNAME column
df_LEIE.drop('BUSNAME', axis=1, inplace=True)

# recode middle name, use only initial
df_LEIE.MIDNAME = df_LEIE.MIDNAME.str.strip().str[0]

# for string vars, replace NaN with ''
loc1 = df_LEIE.columns.get_loc('UPIN')
loc2 = df_LEIE.columns.get_loc('ADDRESS')
loc3 = df_LEIE.columns.get_loc('STATE')
list1 = df_LEIE.columns[:loc1+1].tolist()
list2 = df_LEIE.columns[loc2:loc3+1].tolist()
fill_col = list1 + list2 + ['EXCLTYPE'] + ['WVRSTATE']
for col in fill_col:
    df_LEIE[col].fillna('', inplace=True)

# create a name variable containing both last and first name
df_LEIE['NAME'] = df_LEIE.LASTNAME + ' ' + df_LEIE.FIRSTNAME
df_LEIE.NAME = df_LEIE.NAME.str.strip()

# convert var type
for col in ['NPI', 'ZIP']:
    df_LEIE[col] = np.where(df_LEIE[col] == 0, '', df_LEIE[col].astype(str))

df_LEIE.DOB = np.where(df_LEIE.DOB.isna(), np.datetime64('NaT'), df_LEIE.DOB.astype(str).str[:8].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d')))
for col in ['EXCLDATE', 'REINDATE', 'WAIVERDATE']:
    df_LEIE[col] = np.where(df_LEIE[col] == 0, np.datetime64('NaT'), df_LEIE[col].astype(str).apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')))

# create a flag variable indicating whether a provider is considered as a fraudulent case
# because the time limit of the fraud cases is 5 years, though this list included all fraudulent providers
df_LEIE['fraud_flag'] = np.where(df_LEIE['EXCLDATE'].dt.year.isin(range(2010, 2015)), 1, 0)

print(df_LEIE.shape)
df_LEIE.head(3)

(1519, 20)


,LASTNAME,FIRSTNAME,MIDNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,p_type,NAME,fraud_flag
378,BLANKENSHIP'S PHARMACY,,,PHARMACY,,,,NaT,106 HAYES STREET,CROSSVILLE,TN,33873,1128a1,1988-10-11,NaT,NaT,,O,BLANKENSHIP'S PHARMACY,0
601,CLINICAL LABORATORY ASSOCIATES,,,OTHER BUSINESS,LABORATORY,,,NaT,124 E DIVISION RD,OAK RIDGE,TN,37830,1128a1,1987-04-29,NaT,NaT,,O,CLINICAL LABORATORY ASSOCIATES,0
608,COALFIELD PHARMACY,,,PHARMACY,,,,NaT,,COALFIELD,TN,37830,1128a1,1988-06-24,NaT,NaT,,O,COALFIELD PHARMACY,0


In [5]:
# check the frequency of fraud cases
df_LEIE.fraud_flag.value_counts()

0    1050
1     469
Name: fraud_flag, dtype: int64

In [6]:
# list rows with the same NPI, excluding ''
npi_counts = df_LEIE.NPI.value_counts()
npi_counts = pd.DataFrame(npi_counts)
npi_counts.reset_index(inplace=True)
npi_counts.columns = ['NPI', 'counts']
dup_npi = npi_counts.loc[(npi_counts.NPI != '') & (npi_counts.counts > 1)]['NPI']
df_LEIE[df_LEIE.NPI.isin(dup_npi)]

,LASTNAME,FIRSTNAME,MIDNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,p_type,NAME,fraud_flag
10417,BRILEY,JOHN,M,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,,1356339055,1969-08-28,"1385 S HIGHLAND AVE, STE B1",JACKSON,TN,38301,1128b4,2015-06-18,NaT,NaT,,I,BRILEY JOHN,0
10418,BRILEY,JOHN,M,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,R96352,1356339055,1969-08-28,22 WATLING ROAD,JACKSON,TN,38301,1128a1,2018-04-19,NaT,NaT,,I,BRILEY JOHN,0


In [7]:
# remove duplicates
sortlist = ['NPI', 'STATE', 'CITY', 'LASTNAME', 'FIRSTNAME', 'DOB', 'fraud_flag', 'EXCLDATE']
droplist = ['NPI', 'STATE', 'CITY', 'LASTNAME', 'FIRSTNAME', 'DOB', 'fraud_flag']
#df_LEIE['total_fraud_count'] = df_LEIE.groupby()
df_LEIE.sort_values(by=sortlist, ascending=False, na_position='last', inplace=True)
df_LEIE_nodup = df_LEIE.drop_duplicates(droplist, keep='first')
df_LEIE_nodup.shape

(1511, 20)

# 2. Clean 2014 Provider data


---


> Only using TN data in 2014




In [8]:
# load provider data for TN
df_p2014_TN = pd.read_csv('/content/drive/My Drive/Data Science/Springboard/Data/Provider2014_TN.csv')
df_p2014_TN.head(3)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
0,1871575043,AMBULATORY SURGERY CENTER OF COOL SPRINGS LLC,NaN,NaN,NaN,NaN,O,2009 MALLORY LN,SUITE 100,FRANKLIN,370672845,TN,US,Ambulatory Surgical Center,Y,F,0191T,Internal insertion of eye fluid drainage device,N,26.0,17,26,1605.830,8264.0,1258.970000,1315.470000
1,1134126097,PHYSICIAN'S SURGERY CENTER OF KNOXVILLE LLC,NaN,NaN,NaN,NaN,O,1819 CLINCH AVE,STE 206,KNOXVILLE,379162435,TN,US,Ambulatory Surgical Center,Y,F,15823,Removal of excessive skin and fat of upper eyelid,N,30.0,16,16,462.150,2391.0,362.330000,415.702000
2,1942414750,"HAMILTON EYE INSTITUTE SURGERY CENTER, L.P.",NaN,NaN,NaN,NaN,O,930 MADISON AVE,3RD FLOOR,MEMPHIS,381033410,TN,US,Ambulatory Surgical Center,Y,F,15823,Removal of excessive skin and fat of upper eyelid,N,90.0,48,48,504.728,2327.5,390.148889,409.981778


In [9]:
df_p2014_TN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255311 entries, 0 to 255310
Data columns (total 26 columns):
npi                                 255311 non-null int64
nppes_provider_last_org_name        255311 non-null object
nppes_provider_first_name           245918 non-null object
nppes_provider_mi                   207922 non-null object
nppes_credentials                   242455 non-null object
nppes_provider_gender               245918 non-null object
nppes_entity_code                   255311 non-null object
nppes_provider_street1              255311 non-null object
nppes_provider_street2              125575 non-null object
nppes_provider_city                 255311 non-null object
nppes_provider_zip                  255311 non-null int64
nppes_provider_state                255311 non-null object
nppes_provider_country              255311 non-null object
provider_type                       255311 non-null object
medicare_participation_indicator    255311 non-null object
place_

In [10]:
# exlcude records for drug prescriptions AND select providers that participate in medicare
df_p2014_TN = df_p2014_TN[(df_p2014_TN.hcpcs_drug_indicator == 'N') & (df_p2014_TN.medicare_participation_indicator == 'Y')]

# for string vars, replace NaN with ''
loc1 = df_p2014_TN.columns.get_loc('nppes_provider_last_org_name')
loc2 = df_p2014_TN.columns.get_loc('nppes_provider_city')
loc3 = df_p2014_TN.columns.get_loc('nppes_provider_state')
loc4 = df_p2014_TN.columns.get_loc('hcpcs_drug_indicator')
list1p = df_p2014_TN.columns[loc1:loc2+1].tolist()
list2p = df_p2014_TN.columns[loc3:loc4+1].tolist()
col_list_p = list1p + list2p
for col in col_list_p:
    df_p2014_TN[col].fillna('', inplace=True)

# recode middle name, use only initial
df_p2014_TN.nppes_provider_mi = np.where(df_p2014_TN.nppes_provider_mi.isin(['.', '(']), np.NaN, df_p2014_TN.nppes_provider_mi)

# create a name variable containing first and last name
df_p2014_TN['nppes_name'] = df_p2014_TN.nppes_provider_last_org_name + ' ' + df_p2014_TN.nppes_provider_first_name
df_p2014_TN.nppes_name = df_p2014_TN.nppes_name.str.strip()

# recode npi code
df_p2014_TN.npi = df_p2014_TN.npi.astype(str)

# recode zip code
df_p2014_TN.nppes_provider_zip = df_p2014_TN.nppes_provider_zip.astype(str).str[:5]

# add a variable indicating which year
df_p2014_TN['year'] = '2014'

# check the unique providers using npi
print('The number of unique providers (npi) is ' + str(len(df_p2014_TN.npi.unique().tolist())))

The number of unique providers (npi) is 21642


In [11]:
# check duplicates
p2014_droplist = ['npi', 'hcpcs_code', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_medicare_allowed_amt', 'average_submitted_chrg_amt', 'average_medicare_payment_amt', 'average_medicare_standard_amt']
df_p2014_TN_nodup = df_p2014_TN.drop_duplicates(p2014_droplist, keep='first')
df_p2014_TN_nodup.shape

(236000, 28)

In [12]:
# aggregate the numeric columns on categorical coulumns => each npi has one row of data
# Note: 'place_of_service' indicates facility or office. In general, facility fees are lower than office.
# However, to get one row for each unique npi, I disregarded the difference between the two.
aggre_var = ['line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_medicare_allowed_amt', 'average_submitted_chrg_amt', 
             'average_medicare_payment_amt', 'average_medicare_standard_amt']
group_var = ['provider_type', 'npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name', 'nppes_name', 'nppes_provider_mi', 
             'nppes_provider_state', 'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_street1', 'nppes_provider_street2', 'nppes_entity_code', 
             'nppes_provider_gender', 'nppes_credentials', 'year']
aggregations = {
    'line_srvc_cnt': {
        'line_srvc_cnt_count': 'count', 
        'line_srvc_cnt_sum': 'sum', 
        'line_srvc_cnt_min': 'min', 
        'line_srvc_cnt_mean': 'mean', 
        'line_srvc_cnt_median': 'median',
        'line_srvc_cnt_std': 'std',
        'line_srvc_cnt_max': 'max'
    },
    'bene_unique_cnt': {
        'bene_unique_cnt_count': 'count',
        'bene_unique_cnt_sum': 'sum',
        'bene_unique_cnt_min': 'min',
        'bene_unique_cnt_mean': 'mean',
        'bene_unique_cnt_median': 'median',
        'bene_unique_cnt_std': 'std',
        'bene_unique_cnt_max': 'max'
    },
    'bene_day_srvc_cnt': {
        'bene_day_srvc_cnt_count': 'count',
        'bene_day_srvc_cnt_sum': 'sum',
        'bene_day_srvc_cnt_min': 'min',
        'bene_day_srvc_cnt_mean': 'mean',
        'bene_day_srvc_cnt_median': 'median',
        'bene_day_srvc_cnt_std': 'std',
        'bene_day_srvc_cnt_max': 'max'
    },
    'average_medicare_allowed_amt': {
        'average_medicare_allowed_amt_count': 'count',
        'average_medicare_allowed_amt_sum': 'sum',
        'average_medicare_allowed_amt_min': 'min',
        'average_medicare_allowed_amt_mean': 'mean',
        'average_medicare_allowed_amt_median': 'median',
        'average_medicare_allowed_amt_std': 'std',
        'average_medicare_allowed_amt_max': 'max'
    },
    'average_submitted_chrg_amt': {
        'average_submitted_chrg_amt_count': 'count',
        'average_submitted_chrg_amt_sum': 'sum',
        'average_submitted_chrg_amt_min': 'min',
        'average_submitted_chrg_amt_mean': 'mean',
        'average_submitted_chrg_amt_median': 'median',
        'average_submitted_chrg_amt_std': 'std',
        'average_submitted_chrg_amt_max': 'max'
    },
    'average_medicare_payment_amt': {
        'average_medicare_payment_amt_count': 'count',
        'average_medicare_payment_amt_sum': 'sum',
        'average_medicare_payment_amt_min': 'min',
        'average_medicare_payment_amt_mean': 'mean',
        'average_medicare_payment_amt_median': 'median',
        'average_medicare_payment_amt_std': 'std',
        'average_medicare_payment_amt_max': 'max'
    },
    'average_medicare_standard_amt': {
        'average_medicare_standard_amt_count': 'count',
        'average_medicare_standard_amt_sum': 'sum',
        'average_medicare_standard_amt_min': 'min',
        'average_medicare_standard_amt_mean': 'mean',
        'average_medicare_standard_amt_median': 'median',
        'average_medicare_standard_amt_std': 'std',
        'average_medicare_standard_amt_max': 'max'
    }
}

# fill na before
df_p2014_TN_agg = df_p2014_TN_nodup.groupby(group_var, as_index=False)[aggre_var].agg(aggregations).fillna(0)
df_p2014_TN_agg.info()

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21640 entries, 0 to 21639
Data columns (total 64 columns):
provider_type                           21640 non-null object
npi                                     21640 non-null object
nppes_provider_last_org_name            21640 non-null object
nppes_provider_first_name               21640 non-null object
nppes_name                              21640 non-null object
nppes_provider_mi                       21640 non-null object
nppes_provider_state                    21640 non-null object
nppes_provider_city                     21640 non-null object
nppes_provider_zip                      21640 non-null object
nppes_provider_street1                  21640 non-null object
nppes_provider_street2                  21640 non-null object
nppes_entity_code                       21640 non-null object
nppes_provider_gender                   21640 non-null object
nppes_credentials                       21640 non-null object
year                   

In [13]:
df_p2014_TN_agg.head()

,provider_type,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_name,nppes_provider_mi,nppes_provider_state,nppes_provider_city,nppes_provider_zip,nppes_provider_street1,nppes_provider_street2,nppes_entity_code,nppes_provider_gender,nppes_credentials,year,line_srvc_cnt_count,line_srvc_cnt_sum,line_srvc_cnt_min,line_srvc_cnt_mean,line_srvc_cnt_median,line_srvc_cnt_std,line_srvc_cnt_max,bene_unique_cnt_count,bene_unique_cnt_sum,bene_unique_cnt_min,bene_unique_cnt_mean,bene_unique_cnt_median,bene_unique_cnt_std,bene_unique_cnt_max,bene_day_srvc_cnt_count,bene_day_srvc_cnt_sum,bene_day_srvc_cnt_min,bene_day_srvc_cnt_mean,bene_day_srvc_cnt_median,bene_day_srvc_cnt_std,bene_day_srvc_cnt_max,average_medicare_allowed_amt_count,average_medicare_allowed_amt_sum,average_medicare_allowed_amt_min,average_medicare_allowed_amt_mean,average_medicare_allowed_amt_median,average_medicare_allowed_amt_std,average_medicare_allowed_amt_max,average_submitted_chrg_amt_count,average_submitted_chrg_amt_sum,average_submitted_chrg_amt_min,average_submitted_chrg_amt_mean,average_submitted_chrg_amt_median,average_submitted_chrg_amt_std,average_submitted_chrg_amt_max,average_medicare_payment_amt_count,average_medicare_payment_amt_sum,average_medicare_payment_amt_min,average_medicare_payment_amt_mean,average_medicare_payment_amt_median,average_medicare_payment_amt_std,average_medicare_payment_amt_max,average_medicare_standard_amt_count,average_medicare_standard_amt_sum,average_medicare_standard_amt_min,average_medicare_standard_amt_mean,average_medicare_standard_amt_median,average_medicare_standard_amt_std,average_medicare_standard_amt_max
0,Addiction Medicine,1003998329,JONES,DOUGLAS,JONES DOUGLAS,E,TN,MEMPHIS,38119,6005 PARK AVENUE,SUITE 307,I,M,M.D.,2014,6,687.0,47.0,114.500000,59.0,92.755054,248.0,6,225,24,37.500000,35.5,14.321313,61,6,687,47,114.500000,59.0,92.755054,248,6,519.860000,19.840000,86.643333,97.225000,39.015786,135.610000,6,750.000000,35.000000,125.000000,135.000000,54.129474,175.000000,6,405.379547,19.440000,67.563258,76.225000,27.956777,101.411690,6,427.409028,19.44,71.234838,80.053687,30.180526,108.148082
1,Addiction Medicine,1760574610,ARNETT,DARRELL,ARNETT DARRELL,G,TN,NASHVILLE,37219,1010 4TH AVE N,,I,M,MD,2014,12,1138.0,13.0,94.833333,28.0,160.641234,461.0,12,463,11,38.583333,23.0,38.241953,127,12,1133,11,94.416667,28.0,160.870619,461,12,834.257586,2.780000,69.521466,45.546486,65.524898,193.070000,12,1452.853652,15.000000,121.071138,88.118925,88.604232,289.375000,12,570.644733,2.720000,47.553728,19.634186,43.727651,117.924375,12,623.747840,4.78,51.978987,21.908378,49.154966,138.769375
2,All Other Suppliers,1003166943,TRICARE PHARMACY LLC,,TRICARE PHARMACY LLC,,TN,GALLATIN,37066,179 HANCOCK ST STE 301,,O,,,2014,1,24.0,24.0,24.000000,24.0,0.000000,24.0,1,23,23,23.000000,23.0,0.000000,23,1,24,24,24.000000,24.0,0.000000,24,1,13.262500,13.262500,13.262500,13.262500,0.000000,13.262500,1,14.129167,14.129167,14.129167,14.129167,0.000000,14.129167,1,12.996667,12.996667,12.996667,12.996667,0.000000,12.996667,1,24.580000,24.58,24.580000,24.580000,0.000000,24.580000
3,All Other Suppliers,1184727034,"HARRIS TEETER, LLC",,"HARRIS TEETER, LLC",,TN,NASHVILLE,37205,6002 HIGHWAY #100,,O,,,2014,1,68.0,68.0,68.000000,68.0,0.000000,68.0,1,68,68,68.000000,68.0,0.000000,68,1,68,68,68.000000,68.0,0.000000,68,1,23.007059,23.007059,23.007059,23.007059,0.000000,23.007059,1,29.987059,29.987059,29.987059,29.987059,0.000000,29.987059,1,22.547059,22.547059,22.547059,22.547059,0.000000,22.547059,1,24.580000,24.58,24.580000,24.580000,0.000000,24.580000
4,All Other Suppliers,1184977282,"SALE CREEK PHARMACY, INC",,"SALE CREEK PHARMACY, INC",,TN,SALE CREEK,37373,14821 DAYTON PIKE,SUITE 115,O,,,2014,1,15.0,15.0,15.000000,15.0,0.000000,15.0,1,14,14,14.000000,14.0,0.000000,14,1,15,15,15.000000,15.0,0.000000,15,1,23.010000,23.010000,23.010000,23.010000,0.000000,23.010000,1,23.099333,23.099333,23.099333,23.099333,0.000000,23.099333,1,22.550000,22.550000,22.550

# 3. Merge

In [14]:
# merge the rows with npi values
df_common = pd.merge(df_LEIE_nodup, df_p2014_TN_agg, left_on='NPI', right_on='npi', how='inner')
df_common.shape

(13, 84)

In [15]:
# extract the exclusion list data, which excluded common rows with provider data
df_LEIE_rest = pd.merge(df_LEIE_nodup, df_p2014_TN_agg[['npi']], left_on='NPI', right_on='npi', how='left', indicator=True)
df_LEIE_rest = df_LEIE_rest[df_LEIE_rest['_merge'] == 'left_only']
df_LEIE_rest.drop(['npi', '_merge'], axis=1, inplace=True)
print(df_LEIE_rest.shape)
df_LEIE_rest.head()

(1498, 20)


,LASTNAME,FIRSTNAME,MIDNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,p_type,NAME,fraud_flag
1,BENNE,MICHAEL,,"PHYSICIAN (MD, DO)",GENERAL PRACTICE,,1982786471,1939-02-21,368 PARKWOOD CIRCLE,GREENBACK,TN,37742,1128b4,2016-07-20,NaT,NaT,,I,BENNE MICHAEL,0
2,DOUGLAS,KELVIN,L,"MEDICAL PRACTICE, MD",FAMILY PRACTICE,G01874,1982643482,1953-12-17,106 RIVERS STREET,JACKSON,TN,38301,1128a3,2009-06-18,NaT,NaT,,I,DOUGLAS KELVIN,0
3,DUBE,MICHAEL,N,"PHYSICIAN (MD, DO)",INTERNAL MEDICINE,G30848,1932185121,1961-04-04,113 STERLING SPRINGS DR,JOHNSON CITY,TN,37604,1128a3,2012-07-19,NaT,NaT,,I,DUBE MICHAEL,1
4,LARMORE,CHARLES,R,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,,1922048156,1948-03-27,6835 VILLAGE LAKE CIRCLE,CHATTANOOGA,TN,37412,1128b4,2015-05-20,NaT,NaT,,I,LARMORE CHARLES,0
5,COLES,ROBERT,A,"PHYSICIAN (MD, DO)",GENERAL PRACTICE,,1912938291,1954-09-08,10075 LARKSPUR DRIVE,OOLTEWAH,TN,37363,1128b4,2015-09-20,NaT,NaT,,I,COLES ROBERT,0


In [16]:
# extract the provider data, which excluded common rows with exclusion list data
df_p2014_TN_rest = pd.merge(df_LEIE_nodup[['NPI']], df_p2014_TN_agg, left_on='NPI', right_on='npi', how='right', indicator=True)
df_p2014_TN_rest = df_p2014_TN_rest[df_p2014_TN_rest['_merge'] == 'right_only']
df_p2014_TN_rest.drop(['NPI', '_merge'], axis=1, inplace=True)
print(df_p2014_TN_rest.shape)
df_p2014_TN_rest.head()

(21627, 64)


,provider_type,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_name,nppes_provider_mi,nppes_provider_state,nppes_provider_city,nppes_provider_zip,nppes_provider_street1,nppes_provider_street2,nppes_entity_code,nppes_provider_gender,nppes_credentials,year,line_srvc_cnt_count,line_srvc_cnt_sum,line_srvc_cnt_min,line_srvc_cnt_mean,line_srvc_cnt_median,line_srvc_cnt_std,line_srvc_cnt_max,bene_unique_cnt_count,bene_unique_cnt_sum,bene_unique_cnt_min,bene_unique_cnt_mean,bene_unique_cnt_median,bene_unique_cnt_std,bene_unique_cnt_max,bene_day_srvc_cnt_count,bene_day_srvc_cnt_sum,bene_day_srvc_cnt_min,bene_day_srvc_cnt_mean,bene_day_srvc_cnt_median,bene_day_srvc_cnt_std,bene_day_srvc_cnt_max,average_medicare_allowed_amt_count,average_medicare_allowed_amt_sum,average_medicare_allowed_amt_min,average_medicare_allowed_amt_mean,average_medicare_allowed_amt_median,average_medicare_allowed_amt_std,average_medicare_allowed_amt_max,average_submitted_chrg_amt_count,average_submitted_chrg_amt_sum,average_submitted_chrg_amt_min,average_submitted_chrg_amt_mean,average_submitted_chrg_amt_median,average_submitted_chrg_amt_std,average_submitted_chrg_amt_max,average_medicare_payment_amt_count,average_medicare_payment_amt_sum,average_medicare_payment_amt_min,average_medicare_payment_amt_mean,average_medicare_payment_amt_median,average_medicare_payment_amt_std,average_medicare_payment_amt_max,average_medicare_standard_amt_count,average_medicare_standard_amt_sum,average_medicare_standard_amt_min,average_medicare_standard_amt_mean,average_medicare_standard_amt_median,average_medicare_standard_amt_std,average_medicare_standard_amt_max
13,Addiction Medicine,1003998329,JONES,DOUGLAS,JONES DOUGLAS,E,TN,MEMPHIS,38119,6005 PARK AVENUE,SUITE 307,I,M,M.D.,2014,6,687.0,47.0,114.500000,59.0,92.755054,248.0,6,225,24,37.500000,35.5,14.321313,61,6,687,47,114.500000,59.0,92.755054,248,6,519.860000,19.840000,86.643333,97.225000,39.015786,135.610000,6,750.000000,35.000000,125.000000,135.000000,54.129474,175.000000,6,405.379547,19.440000,67.563258,76.225000,27.956777,101.411690,6,427.409028,19.44,71.234838,80.053687,30.180526,108.148082
14,Addiction Medicine,1760574610,ARNETT,DARRELL,ARNETT DARRELL,G,TN,NASHVILLE,37219,1010 4TH AVE N,,I,M,MD,2014,12,1138.0,13.0,94.833333,28.0,160.641234,461.0,12,463,11,38.583333,23.0,38.241953,127,12,1133,11,94.416667,28.0,160.870619,461,12,834.257586,2.780000,69.521466,45.546486,65.524898,193.070000,12,1452.853652,15.000000,121.071138,88.118925,88.604232,289.375000,12,570.644733,2.720000,47.553728,19.634186,43.727651,117.924375,12,623.747840,4.78,51.978987,21.908378,49.154966,138.769375
15,All Other Suppliers,1003166943,TRICARE PHARMACY LLC,,TRICARE PHARMACY LLC,,TN,GALLATIN,37066,179 HANCOCK ST STE 301,,O,,,2014,1,24.0,24.0,24.000000,24.0,0.000000,24.0,1,23,23,23.000000,23.0,0.000000,23,1,24,24,24.000000,24.0,0.000000,24,1,13.262500,13.262500,13.262500,13.262500,0.000000,13.262500,1,14.129167,14.129167,14.129167,14.129167,0.000000,14.129167,1,12.996667,12.996667,12.996667,12.996667,0.000000,12.996667,1,24.580000,24.58,24.580000,24.580000,0.000000,24.580000
16,All Other Suppliers,1184727034,"HARRIS TEETER, LLC",,"HARRIS TEETER, LLC",,TN,NASHVILLE,37205,6002 HIGHWAY #100,,O,,,2014,1,68.0,68.0,68.000000,68.0,0.000000,68.0,1,68,68,68.000000,68.0,0.000000,68,1,68,68,68.000000,68.0,0.000000,68,1,23.007059,23.007059,23.007059,23.007059,0.000000,23.007059,1,29.987059,29.987059,29.987059,29.987059,0.000000,29.987059,1,22.547059,22.547059,22.547059,22.547059,0.000000,22.547059,1,24.580000,24.58,24.580000,24.580000,0.000000,24.580000
17,All Other Suppliers,1184977282,"SALE CREEK PHARMACY, INC",,"SALE CREEK PHARMACY, INC",,TN,SALE CREEK,37373,14821 DAYTON PIKE,SUITE 115,O,,,2014,1,15.0,15.0,15.000000,15.0,0.000000,15.0,1,14,14,14.000000,14.0,0.000000,14,1,15,15,15.000000,15.0,0.000000,15,1,23.010000,23.010000,23.010000,23.010000,0.000000,23.010000,1,23.099333,23.099333,23.099333,23.099333,0.000000,23.099333,1,22.550000,22.550000,2

In [17]:
# create a blocking variable in each dataset (state + first letter of city)
# check whether blocking variables in two datasets are the same
df_LEIE_rest['l_grpvar'] = df_LEIE_rest.STATE + ' ' + df_LEIE_rest.CITY.str[0]
comb_len1 = len(df_LEIE_rest['l_grpvar'].unique().tolist())

df_p2014_TN_rest['p_grpvar'] = df_p2014_TN_rest.nppes_provider_state + ' ' + df_p2014_TN_rest.nppes_provider_city.str[0]
comb_len2 = len(df_p2014_TN_rest['p_grpvar'].unique().tolist())

# compare two lists
if comb_len1 == comb_len2:
    print('True: The blocking variables from the two datasets have the same number of combination groups (state + first letter of city)')
    print(comb_len1)
else:
    print('False: The blocking variables from the two datasets have the same number of combination groups (state + first letter of city)')
    print('exclusion list: ', str(comb_len1), 'provider data: ', str(comb_len2))

l_gpname = sorted(df_LEIE_rest['l_grpvar'].unique())
p_gpname = sorted(df_p2014_TN_rest['p_grpvar'].unique())
comp = zip(l_gpname, p_gpname)
list(comp)

True: The blocking variables from the two datasets have the same number of combination groups (state + first letter of city)
21


[('TN A', 'TN A'),
 ('TN B', 'TN B'),
 ('TN C', 'TN C'),
 ('TN D', 'TN D'),
 ('TN E', 'TN E'),
 ('TN F', 'TN F'),
 ('TN G', 'TN G'),
 ('TN H', 'TN H'),
 ('TN J', 'TN J'),
 ('TN K', 'TN K'),
 ('TN L', 'TN L'),
 ('TN M', 'TN M'),
 ('TN N', 'TN N'),
 ('TN O', 'TN O'),
 ('TN P', 'TN P'),
 ('TN R', 'TN R'),
 ('TN S', 'TN S'),
 ('TN T', 'TN T'),
 ('TN U', 'TN U'),
 ('TN V', 'TN V'),
 ('TN W', 'TN W')]

In [18]:
df_p2014_TN_rest.groupby(['p_grpvar']).get_group('TN A').shape

(306, 65)

In [19]:
# fuzzy string matching
def fuzz_match(key, choices, scorer, cutoff):
    return process.extractOne(key, choices=choices, scorer=scorer, score_cutoff=cutoff)

result = []
for gp1 in l_gpname:
    l_gp = df_LEIE_rest.groupby(['l_grpvar']).get_group(gp1)
    #l_gp_start = l_gp.index.min()
    #l_gp_end = l_gp.index.max()
    for gp2 in p_gpname:
        p_gp = df_p2014_TN_rest.groupby(['p_grpvar']).get_group(gp2)
        #p_gp_start = p_gp.index.min()
        #p_gp_end = p_gp.index.max()
        if gp1 == gp2:
            new_result = l_gp['NAME'].apply(fuzz_match, args=(p_gp['nppes_name'], fuzz.ratio, 80))
            result.append(new_result)

result

[16                         None
 51                         None
 54                         None
 1470                       None
 1471                       None
 1472                       None
 1473                       None
 1474                       None
 1475                       None
 1476                       None
 1477                       None
 1478                       None
 1479                       None
 1480                       None
 1481                       None
 1482                       None
 1483                       None
 1484                       None
 1485                       None
 1486                       None
 1487                       None
 1488                       None
 1489    (RHEA ROBERT, 83, 7728)
 1490                       None
 1491                       None
 1492                       None
 1493                       None
 1494                       None
 1495                       None
 1496                       None
 1497     

In [21]:
type(new_result)

pandas.core.series.Series

In [0]:
df = pd.DataFrame({'LEIE_index': new_result.index, 'result_temp': new_result.values})

In [24]:
df

,LEIE_index,result_temp
0,69,None
1,70,None
2,71,None
3,72,None
4,73,None
5,74,None
6,75,None
7,76,None
8,77,"(ROBERTSON TASMA, 84, 12706)"
9,78,None


In [28]:
df_LEIE_rest.iloc[1489]

LASTNAME                    TOMLIN
FIRSTNAME                    JULIE
MIDNAME                          A
GENERAL       IND- LIC HC SERV PRO
SPECIALTY                THERAPIST
UPIN                              
NPI                               
DOB            1977-05-24 00:00:00
ADDRESS           314 DIXON AVENUE
CITY                        ATHENS
STATE                           TN
ZIP                          37303
EXCLTYPE                    1128b4
EXCLDATE       2014-06-19 00:00:00
REINDATE                       NaT
WAIVERDATE                     NaT
WVRSTATE                          
p_type                           I
NAME                  TOMLIN JULIE
fraud_flag                       1
l_grpvar                      TN A
Name: 1471, dtype: object

In [30]:
df_p2014_TN_rest.iloc[7728]

provider_type                           Family Practice
npi                                          1962614552
nppes_provider_last_org_name                      DUFFY
nppes_provider_first_name                        JAMIRA
nppes_name                                 DUFFY JAMIRA
nppes_provider_mi                                     N
nppes_provider_state                                 TN
nppes_provider_city                        MURFREESBORO
nppes_provider_zip                                37130
nppes_provider_street1                  726 S CHURCH ST
nppes_provider_street2                                 
nppes_entity_code                                     I
nppes_provider_gender                                 F
nppes_credentials                                  M.D.
year                                               2014
line_srvc_cnt_count                                  22
line_srvc_cnt_sum                                  2830
line_srvc_cnt_min                               